In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

In [3]:
np.set_printoptions(threshold=sys.maxsize)

# Capa Dropout

Dropout actúa como una técnica de _regularización_ que elimina o desactiva nodos en la propagación hacia adelante, lo que hace que la red sea menos propensa al sobreajuste al evitar que la red dependa demasiado de cualquier neurona individual. Esta capa no tiene parámetros.

* En la propagación hacia adelante, las entradas se establecen en cero con una probabilidad $p$, y de lo contrario se escalan por $\frac{1}{1-p}$.

  - La propagación hacia adelante durante el entrenamiento solo se utiliza para configurar la red para la propagación hacia atrás, donde la red se modifica realmente.
  
  - Para cada neurona individual en la capa, podemos decir que $x \sim B(1, p)$, ya que estamos considerando un solo "experimento" (la activación o desactivación de la neurona) con una probabilidad de éxito de $p$.

* En la propagación hacia atrás, los gradientes para las mismas unidades eliminadas se anulan; otros gradientes se escalan por el mismo factor de $\frac{1}{1-p}$.

  - Es decir, si un nodo fue eliminado por la capa Dropout, entonces su influencia (el gradiente) en los pesos salientes es también 0 (ya que $0 * w_i = 0$). En resumen, la propagación hacia atrás funciona como siempre.


> NOTA: tener en cuenta que durante la fase de prueba o validación, todas las neuronas están activas (es decir, no se aplica Dropout) para obtener una predicción basada en toda la red.

In [4]:
np.random.seed(123)

din=10
batch_size=2

x = np.random.rand(batch_size,din)
                       
layer=nn.Dropout(p=0.5)

In [5]:
x

array([[0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
        0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752],
       [0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
        0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759]])

In [6]:
y = layer.forward(x)
y, y.shape

(array([[1.39293837, 0.57227867, 0.45370291, 1.10262954, 1.43893794,
         0.        , 0.        , 0.        , 0.        , 0.78423504],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.35090351, 1.06310275, 1.06365517]]),
 (2, 10))

In [7]:
# Define el gradiente de la salida
g = np.random.rand(*y.shape)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

(array([[1.2479059 , 0.23123679, 0.63457096, 0.82965242, 1.73261832,
         0.        , 0.        , 0.        , 0.        , 1.22578905],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 1.36260153, 1.75091368, 1.02084467]]),
 {})

In [8]:
from edunn.model import Phase
layer.set_phase(Phase.Test)
y = layer.forward(x)
y, y.shape

(array([[0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
         0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752],
        [0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
         0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759]]),
 (2, 10))

## Comprobaciones con PyTorch

In [9]:
import torch
import torch.nn as tnn

x = torch.from_numpy(x).to(torch.float)

# Definimos la capa Dropout
dropout = tnn.Dropout(p=0.5)

x.requires_grad = True

y_torch = dropout(x)
y_torch

tensor([[1.3929, 0.5723, 0.0000, 1.1026, 1.4389, 0.8462, 0.0000, 0.0000, 0.9619,
         0.0000],
        [0.6864, 1.4581, 0.8771, 0.1194, 0.0000, 1.4760, 0.0000, 0.3509, 1.0631,
         0.0000]], grad_fn=<MulBackward0>)

In [10]:
# Define el gradiente de la salida
g = torch.from_numpy(g).to(torch.double)

# Propaga el gradiente hacia atrás
x.grad = None  # Limpiamos los gradientes existentes
y_torch.backward(g)

# Imprime el gradiente de la imagen de entrada
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)

Gradiente de la entrada (δE/δx):
tensor([[1.2479, 0.2312, 0.0000, 0.8297, 1.7326, 0.5009, 0.0000, 0.0000, 1.0390,
         0.0000],
        [0.2413, 1.6527, 1.2061, 1.0901, 0.0000, 0.6082, 0.0000, 1.3626, 1.7509,
         0.0000]]) torch.Size([2, 10])


In [11]:
dropout.eval()
y_torch = dropout(x)
y_torch

tensor([[0.6965, 0.2861, 0.2269, 0.5513, 0.7195, 0.4231, 0.9808, 0.6848, 0.4809,
         0.3921],
        [0.3432, 0.7290, 0.4386, 0.0597, 0.3980, 0.7380, 0.1825, 0.1755, 0.5316,
         0.5318]], requires_grad=True)